# Reading in the data

In [6]:
%load_ext autotime

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import random
# pd.options.display.max_columns
pd.set_option("display.max_colwidth",200)
pd.set_option("display.max_columns",20)
pd.set_option('float_format', '{:.3f}'.format)


The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 82.2 ms


In [8]:
# # Load the usage data
with open('rm_oscillated_cleaned_data.txt') as f:
    lines = f.readlines()#[:100000]
    df = []
    for line in lines:
        line = line.replace('\n', '')
        data = line.split(',')
        df.append(data)
usage = pd.DataFrame(df, columns=['uid','timestamp', 'loc', 'app_id','traffic'])
# output_users(usage)
# usage['traffic'] = usage['traffic'].astype('float64') / 1e6 # Convert traffic to MB
usage['timestamp'] = usage['timestamp'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S')) 
# usage['timestamp'] = usage['timestamp'].apply(lambda x: datetime.strptime(x, "%Y%m%d%H%M%S")) # Convert to datetime object
# usage['timestamp'] = usage['timestamp'].datetime
usage['loc'] = usage['loc'].astype('int64')

# usage = pd.read_csv("App_usage_trace.txt", delimiter=' ', header=None)
# usage.columns = ['uid','timestamp', 'loc', 'app_id','traffic']
# usage['traffic'] = usage['traffic'].astype('float64') / 1e6 # Convert traffic to MB
# usage['timestamp'] = usage['timestamp'].astype(str)
# usage['timestamp'] = usage['timestamp'].apply(lambda x: datetime.strptime(x, "%Y%m%d%H%M%S")) # Convert to datetime object
# usage['loc'] = usage['loc'].astype('int64')

app2cat = pd.read_csv('App2Category.txt', delimiter='\t', header=None)
app2cat.columns = ['app_id','cat_id']
# app2cat.apply(pd.to_numeric)
# app2cat = pd.DataFrame(df, columns=['app_id','cat_id'])

# Load base station POI data
base_poi = pd.read_csv("base_poi.txt", delimiter='\t')

# Load Category dictionary
cat = pd.read_csv("Categorys.txt", delimiter='\t', header = None)
cat.columns = ['cat_id','category']
cat.set_index('cat_id', inplace=True)


print("< ---- Basic information ---- >")
print("- {} base stations. {} entries".format(base_poi.shape[0], usage.shape[0]))
print()
print("< ---- Unique users ---- >")
print("- {} unique users".format(len(usage['uid'].unique())))

print()
print("< ---- Unique apps ---- >")
print("- {} unique users".format(len(usage['app_id'].unique())))



< ---- Basic information ---- >
- 9851 base stations. 3970225 entries

< ---- Unique users ---- >
- 697 unique users

< ---- Unique apps ---- >
- 1675 unique users
time: 1min 33s


In [11]:
base_poi.head()

,loc,Medical care,Hotel,Business affairs,Life service,Transportation hub,Culture,Sports,Residence,Entertainment and leisure,Scenic spot,Government,Factory,Shopping,Restaurant,Education,Landmark,Other
0,0,1,3,4,1,0,0,0,54,0,0,1,0,6,6,2,0,4
1,1,0,0,0,0,0,0,0,27,0,0,1,0,0,0,1,2,0
2,2,0,0,12,0,1,0,0,3,0,0,1,0,1,0,0,0,0
3,3,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,4,0,3,0,0,0,1,0,0,0


time: 53.2 ms


In [10]:
base_poi.rename(columns={'BaseID': 'loc'}, inplace=True)

time: 7.57 ms


In [31]:
base_poi1 = pd.DataFrame({'hot_poi':[i for i in base_poi.values]})

time: 28 ms


In [33]:
base_poi1.head()

,hot_poi
0,"[0, 1, 3, 4, 1, 0, 0, 0, 54, 0, 0, 1, 0, 6, 6, 2, 0, 4]"
1,"[1, 0, 0, 0, 0, 0, 0, 0, 27, 0, 0, 1, 0, 0, 0, 1, 2, 0]"
2,"[2, 0, 0, 12, 0, 1, 0, 0, 3, 0, 0, 1, 0, 1, 0, 0, 0, 0]"
3,"[3, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
4,"[4, 0, 0, 0, 0, 0, 0, 0, 4, 0, 3, 0, 0, 0, 1, 0, 0, 0]"


time: 71.8 ms


In [13]:
usage1 = pd.merge(usage, base_poi, how="left", on=["loc"])

# usage = usage.set_index('loc').join(base_poi.set_index('BaseID'))

time: 3.52 s


In [14]:
usage1.head()

,uid,timestamp,loc,app_id,traffic,Medical care,Hotel,Business affairs,Life service,Transportation hub,...,Residence,Entertainment and leisure,Scenic spot,Government,Factory,Shopping,Restaurant,Education,Landmark,Other
0,0,2016-04-21 08:42:21,8194,342,0.030432,2,2,1,4,0,...,40,7,0,2,0,13,15,1,3,5
1,0,2016-04-21 08:44:12,8194,1,0.008329,2,2,1,4,0,...,40,7,0,2,0,13,15,1,3,5
2,0,2016-04-21 08:44:47,8194,857,0.026662,2,2,1,4,0,...,40,7,0,2,0,13,15,1,3,5
3,0,2016-04-21 08:44:48,8194,857,0.001689,2,2,1,4,0,...,40,7,0,2,0,13,15,1,3,5
4,0,2016-04-21 08:44:49,8194,31,0.008758,2,2,1,4,0,...,40,7,0,2,0,13,15,1,3,5


time: 51.1 ms


In [3]:
app2cat = pd.read_csv('App2Category.txt', delimiter='\t', header=None)
app2cat.columns = ['app_id','cat_id']

time: 5.36 ms


# Targeted Vector will include:
#### Time context
- morning (0600 - 1200)
- afternoon (12:00 - 1800)
- night (1800 - 0000)
- mid-night (0000 - 0600)
#### Space
- POI distribution (embedded into low dimension, such as from 18 to 3)
#### App
- app_id
- app_category
- network usage
- time between current and previous requests

## Adding the time context

In [4]:
# Function to one-hot-code the time to period of the day
def convertToPeriod(i):
    if i >= 0 and i < 6:
        return "midnight"
    elif i >= 6 and i < 12:
        return "morning"
    elif i >= 12 and i < 18:
        return "afternoon"
    elif i >= 18 :
        return "night"

# Create a new column that takes the hour from the timestamp
usage['hour'] = usage['timestamp'].apply(lambda x: x.hour)

# Convert the hour to period
usage['period'] = usage['hour'].apply(lambda x: convertToPeriod(x))

# Get weekday/weekend feature
usage['dayOfWeek'] = usage['timestamp'].apply(lambda x: "weekday" if x.weekday() <= 5  else "weekend")

# Concatenate the dummies and the original df
usage = pd.concat([usage, pd.get_dummies(usage['period']), pd.get_dummies(usage['dayOfWeek'])], axis=1)

time: 53.1 s


In [5]:
usage.head()

,uid,timestamp,loc,app_id,traffic,hour,period,dayOfWeek,afternoon,midnight,morning,night,weekday,weekend
0,0,2016-04-21 08:42:21,8194,342,0.030432,8,morning,weekday,0,0,1,0,1,0
1,0,2016-04-21 08:44:12,8194,1,0.008329,8,morning,weekday,0,0,1,0,1,0
2,0,2016-04-21 08:44:47,8194,857,0.026662,8,morning,weekday,0,0,1,0,1,0
3,0,2016-04-21 08:44:48,8194,857,0.001689,8,morning,weekday,0,0,1,0,1,0
4,0,2016-04-21 08:44:49,8194,31,0.008758,8,morning,weekday,0,0,1,0,1,0


time: 68.6 ms


In [6]:
# app2cat['app_id'] = app2cat['app_id'].astype(str)
# x = pd.merge(usage, app2cat, on='app_id', how='left')
# x.head()

time: 372 µs


# Handling the App Usage Information

In [7]:
# # Define function for getting app category
def find_cat(i):
    return app2cat[app2cat['app_id'] == i].iloc[0]['cat_id']

# # Add the app category columns
app2cat['app_id'] = app2cat['app_id'].astype(str)
usage['app_cat'] = usage['app_id'].apply(lambda x: find_cat(x))
# usage = pd.merge(usage, app2cat, on='app_id', how='left')

# Shift to get delta
usage['previous_request'] = usage['timestamp'].shift(1, fill_value=np.nan)

# Get the delta between current and previous requests
usage['delta'] = usage['timestamp'] - usage['previous_request']

# Convert datetime delta to number of seconds
usage['delta'] = usage['delta'].apply(lambda x: x.total_seconds())

time: 1h 7min 59s


In [8]:
usage.head()

,uid,timestamp,loc,app_id,traffic,hour,period,dayOfWeek,afternoon,midnight,morning,night,weekday,weekend,app_cat,previous_request,delta
0,0,2016-04-21 08:42:21,8194,342,0.030432,8,morning,weekday,0,0,1,0,1,0,0,NaT,nan
1,0,2016-04-21 08:44:12,8194,1,0.008329,8,morning,weekday,0,0,1,0,1,0,4,2016-04-21 08:42:21,111.000
2,0,2016-04-21 08:44:47,8194,857,0.026662,8,morning,weekday,0,0,1,0,1,0,8,2016-04-21 08:44:12,35.000
3,0,2016-04-21 08:44:48,8194,857,0.001689,8,morning,weekday,0,0,1,0,1,0,8,2016-04-21 08:44:47,1.000
4,0,2016-04-21 08:44:49,8194,31,0.008758,8,morning,weekday,0,0,1,0,1,0,8,2016-04-21 08:44:48,1.000


time: 52.9 ms


# Concatenate the POI Distribution

In [9]:
usage = usage.set_index('loc').join(base_poi.set_index('BaseID'))

time: 7.74 s


# Drop unwanted columns

In [10]:
usage.drop(['timestamp', 'hour', 'period', 'previous_request', 'dayOfWeek'], axis=1, inplace=True)

time: 6.38 s


In [11]:
usage.dropna(inplace=True)

time: 2.57 s


# Output as CSV

In [12]:
# usage.to_csv('vectorized_usage_osc_cleans.csv')

time: 338 µs


In [13]:
# usage.to_csv('ectorized_usage.csv')

time: 1.03 ms


In [14]:
usage.columns

Index(['uid', 'app_id', 'traffic', 'afternoon', 'midnight', 'morning', 'night',
       'weekday', 'weekend', 'app_cat', 'delta', 'Medical care', 'Hotel',
       'Business affairs', 'Life service', 'Transportation hub', 'Culture',
       'Sports', 'Residence', 'Entertainment and leisure', 'Scenic spot',
       'Government', 'Factory', 'Shopping', 'Restaurant', 'Education',
       'Landmark', 'Other'],
      dtype='object')

time: 5.84 ms


In [15]:
usage.head()

,uid,app_id,traffic,afternoon,midnight,morning,night,weekday,weekend,app_cat,...,Residence,Entertainment and leisure,Scenic spot,Government,Factory,Shopping,Restaurant,Education,Landmark,Other
0,301,9,0.001615,1,0,0,0,1,0,13,...,54,0,0,1,0,6,6,2,0,4
0,301,9,0.000355,1,0,0,0,1,0,13,...,54,0,0,1,0,6,6,2,0,4
0,301,27,0.005134,1,0,0,0,1,0,0,...,54,0,0,1,0,6,6,2,0,4
0,301,27,0.000291,1,0,0,0,1,0,0,...,54,0,0,1,0,6,6,2,0,4
0,301,27,0.001629,1,0,0,0,1,0,0,...,54,0,0,1,0,6,6,2,0,4


time: 44.1 ms
